In [1]:
from typing import Any, List, Dict, Optional
from pydantic.utils import GetterDict
from pydantic import BaseModel, ValidationError, Field, constr, conint
from xml.etree.ElementTree import fromstring
from sqlalchemy import Column, Integer, String, JSON
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base

In [2]:
class User(BaseModel):
    id: int
    name = "Jane Doe"

In [3]:
user = User(id='123')
user_x = User(id=123.45)

In [4]:
assert user.id == 123
assert user_x.id == 123
assert isinstance(user_x.id, int)

In [5]:
assert user.name == "Jane Doe"

In [6]:
assert user.__fields_set__ == {"id"}

In [7]:
assert user.dict() == dict(user) == {"id": 123, "name": "Jane Doe"}

In [8]:
user.id = 321
assert user.id == 321

In [9]:
class Foo(BaseModel):
    count: int
    size: Optional[float] = None

In [10]:
class Bar(BaseModel):
    apple = "x"
    banana = "y"

In [11]:
class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]

In [12]:
m = Spam(foo={"count": 4}, bars=[{"apple": "x1"}, {"apple": "x2"}])
print(f"{m = }")
print(f"{m.dict() = }")

m = Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])
m.dict() = {'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


In [13]:
Base = declarative_base()

In [14]:
class CompanyOrm(Base):
    __tablename__ = "companies"
    
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))

In [15]:
class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]
    
    class Config:
        orm_mode = True

In [16]:
co_orm = CompanyOrm(
    id=123,
    public_key="foobar",
    name="Testing",
    domains=["example.com", "foobar.com"],
)
print(f"{co_orm = }")

co_orm = <__main__.CompanyOrm object at 0x00000269F7068730>


In [17]:
co_model = CompanyModel.from_orm(co_orm)
print(f"{co_model = }")

co_model = CompanyModel(id=123, public_key='foobar', name='Testing', domains=['example.com', 'foobar.com'])


In [18]:
class SQLModel(Base):
    __tablename__ = "my_table"
    
    id = Column("id", Integer, primary_key=True)
    metadata_ = Column("metadata", JSON)

In [19]:
class ReserveKeyModel(BaseModel):
    metadata: Dict[str, str] = Field(alias="metadata_")
    
    class Config:
        orm_mode = True

In [20]:
sql_model = SQLModel(metadata_={"key": "value"}, id=1)
pydantic_model = ReserveKeyModel.from_orm(sql_model)
print(f"{pydantic_model = }")
print(f"{pydantic_model.dict(by_alias=True) = }")

pydantic_model = ReserveKeyModel(metadata={'key': 'value'})
pydantic_model.dict(by_alias=True) = {'metadata_': {'key': 'value'}}


In [21]:
class PetClass:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species

In [22]:
class PersonClass:
    def __init__(self, *, name: str, age: float = None, pets: List[PetClass]):
        self.name = name
        self.age = age
        self.pets = pets

In [23]:
class Pet(BaseModel):
    name: str
    species: str
    
    class Config:
        orm_mode = True

In [24]:
class Person(BaseModel):
    name: str
    age: float = None
    pets: List[Pet]
    
    class Config:
        orm_mode = True

In [25]:
bones = PetClass(name="Bones", species="dog")
orion = PetClass(name="Orion", species="cat")
anna = PersonClass(name="Anna", age=20, pets=[bones, orion])
anna_model = Person.from_orm(anna)
print(f"{anna_model = }")

anna_model = Person(name='Anna', age=20.0, pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')])


In [26]:
class UserGetter(GetterDict):
    def get(self, key: str, default: Any) -> Any:
        # element attributes
        if key in {"Id", "Status"}:
            return self._obj.attrib.get(key, default)
        # element children
        else:
            try:
                return self._obj.find(key).attrib["Value"]
            except (AttributeError, KeyError):
                return default

In [27]:
class UserGetterModel(BaseModel):
    Id: int
    Status: Optional[str]
    FirstName: Optional[str]
    LastName: Optional[str]
    LoggedIn: bool
    
    class Config:
        orm_mode = True
        getter_dict = UserGetter

In [28]:
xmlstring = """
<User Id="2138">
    <FirstName />
    <LoggedIn Value="true" />
</User>
"""

user = UserGetterModel.from_orm(fromstring(xmlstring))
print(f"{user = }")

user = UserGetterModel(Id=2138, Status=None, FirstName=None, LastName=None, LoggedIn=True)


In [29]:
class Location(BaseModel):
    lat = 0.1
    lng = 10.1

In [30]:
class Model(BaseModel):
    is_required: float
    gt_int: conint(gt=42)
    list_of_ints: List[int] = None
    a_float: float = None
    recursive_model: Location = None

In [31]:
data = dict(
    list_of_ints=["1", 2, "bad"],
    a_float="not a float",
    recursive_model={"lat": 4.2, "lng": "New York"},
    gt_int=21,
)

In [32]:
try:
    print(f"{Model(**data) = }")
except ValidationError as e:
    print(f"{e = }")
    print(f"{str(e) = }")
    print(f"{e.json() = }")
    print(f"{e.errors() = }")
    
    for error in e.errors():
        print(f"{error['loc'] = }, {error['type'] = }, {error['msg'] = }")
        print(f"{error['ctx'] = }") if "ctx" in error else print("{error['ctx'] = None}")

e = ValidationError(model='Model', errors=[{'loc': ('is_required',), 'msg': 'field required', 'type': 'value_error.missing'}, {'loc': ('gt_int',), 'msg': 'ensure this value is greater than 42', 'type': 'value_error.number.not_gt', 'ctx': {'limit_value': 42}}, {'loc': ('list_of_ints', 2), 'msg': 'value is not a valid integer', 'type': 'type_error.integer'}, {'loc': ('a_float',), 'msg': 'value is not a valid float', 'type': 'type_error.float'}, {'loc': ('recursive_model', 'lng'), 'msg': 'value is not a valid float', 'type': 'type_error.float'}])
str(e) = '5 validation errors for Model\nis_required\n  field required (type=value_error.missing)\ngt_int\n  ensure this value is greater than 42 (type=value_error.number.not_gt; limit_value=42)\nlist_of_ints -> 2\n  value is not a valid integer (type=type_error.integer)\na_float\n  value is not a valid float (type=type_error.float)\nrecursive_model -> lng\n  value is not a valid float (type=type_error.float)'
e.json() = '[\n  {\n    "loc": [\n  